## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
# trainLabel = trainL
print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [3]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    model = Sequential() 
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(4, activation = "softmax"))
    
    adam = Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    
    return model


batch_S = [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
parameter_grid = dict(batch_size=batch_S)
model = KerasClassifier(build_fn = create_model, epochs=80, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=kf, verbose = 1)
grid_result = grid.fit(trainData, trainLabel)

print('Finish training!')

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Epoch 1/80
8120/8120 [==============================] - 13s 2ms/step - loss: 1.0003 - acc: 0.5752
Epoch 2/80
8120/8120 [==============================] - 7s 856us/step - loss: 0.8712 - acc: 0.6161
Epoch 3/80
8120/8120 [==============================] - 7s 872us/step - loss: 0.7988 - acc: 0.6607
Epoch 4/80
8120/8120 [==============================] - 7s 860us/step - loss: 0.7118 - acc: 0.7031
Epoch 5/80
8120/8120 [==============================] - 7s 892us/step - loss: 0.6495 - acc: 0.7404
Epoch 6/80
8120/8120 [==============================] - 7s 879us/step - loss: 0.6125 - acc: 0.7642
Epoch 7/80
8120/8120 [==============================] - 7s 863us/step - loss: 0.5777 - acc: 0.7745
Epoch 8/80
8120/8120 [==============================] - 7s 870us/step - loss: 0.5486 - acc: 0.7915
Epoch 9/80
8120/8120 [==============================] - 7s 885us/step - loss: 0.5378 - acc: 0.7993
Epoch 10/80
8120/8120 [==========================

8121/8121 [==============================] - 8s 956us/step - loss: 0.9965 - acc: 0.5832
Epoch 2/80
8121/8121 [==============================] - 7s 879us/step - loss: 0.8971 - acc: 0.6036
Epoch 3/80
8121/8121 [==============================] - 7s 862us/step - loss: 0.8075 - acc: 0.6595
Epoch 4/80
8121/8121 [==============================] - 7s 900us/step - loss: 0.7057 - acc: 0.7069
Epoch 5/80
8121/8121 [==============================] - 7s 867us/step - loss: 0.6656 - acc: 0.7239
Epoch 6/80
8121/8121 [==============================] - 7s 868us/step - loss: 0.6999 - acc: 0.7122
Epoch 7/80
8121/8121 [==============================] - 7s 891us/step - loss: 0.6461 - acc: 0.7348
Epoch 8/80
8121/8121 [==============================] - 7s 883us/step - loss: 0.6016 - acc: 0.7607
Epoch 9/80
8121/8121 [==============================] - 7s 921us/step - loss: 0.5571 - acc: 0.7819
Epoch 10/80
8121/8121 [==============================] - 7s 886us/step - loss: 0.5742 - acc: 0.7811
Epoch 11/80
8121/812

Epoch 2/80
8121/8121 [==============================] - 7s 882us/step - loss: 0.9253 - acc: 0.5876
Epoch 3/80
8121/8121 [==============================] - 7s 889us/step - loss: 0.8183 - acc: 0.6388
Epoch 4/80
8121/8121 [==============================] - 7s 888us/step - loss: 0.7500 - acc: 0.6819
Epoch 5/80
8121/8121 [==============================] - 7s 871us/step - loss: 0.6990 - acc: 0.7122
Epoch 6/80
8121/8121 [==============================] - 7s 886us/step - loss: 0.6885 - acc: 0.7227
Epoch 7/80
8121/8121 [==============================] - 7s 873us/step - loss: 0.6434 - acc: 0.7334
Epoch 8/80
8121/8121 [==============================] - 7s 879us/step - loss: 0.6053 - acc: 0.7497
Epoch 9/80
8121/8121 [==============================] - 7s 895us/step - loss: 0.5783 - acc: 0.7603
Epoch 10/80
8121/8121 [==============================] - 7s 868us/step - loss: 0.5547 - acc: 0.7843
Epoch 11/80
8121/8121 [==============================] - 7s 877us/step - loss: 0.5271 - acc: 0.7968
Epoch 12

8121/8121 [==============================] - 7s 870us/step - loss: 0.9063 - acc: 0.5991
Epoch 3/80
8121/8121 [==============================] - 7s 867us/step - loss: 0.7935 - acc: 0.6594
Epoch 4/80
8121/8121 [==============================] - 7s 887us/step - loss: 0.7053 - acc: 0.7104
Epoch 5/80
8121/8121 [==============================] - 7s 921us/step - loss: 0.6604 - acc: 0.7327
Epoch 6/80
8121/8121 [==============================] - 7s 894us/step - loss: 0.6196 - acc: 0.7466
Epoch 7/80
8121/8121 [==============================] - 7s 889us/step - loss: 0.6199 - acc: 0.7454
Epoch 8/80
8121/8121 [==============================] - 7s 882us/step - loss: 0.5846 - acc: 0.7703
Epoch 9/80
8121/8121 [==============================] - 7s 894us/step - loss: 0.5490 - acc: 0.7822
Epoch 10/80
8121/8121 [==============================] - 7s 916us/step - loss: 0.5298 - acc: 0.7912
Epoch 11/80
8121/8121 [==============================] - 7s 893us/step - loss: 0.5190 - acc: 0.7989
Epoch 12/80
8121/81

Epoch 3/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.8115 - acc: 0.6455
Epoch 4/80
8121/8121 [==============================] - 7s 898us/step - loss: 0.7489 - acc: 0.6849
Epoch 5/80
8121/8121 [==============================] - 7s 905us/step - loss: 0.7292 - acc: 0.6887
Epoch 6/80
8121/8121 [==============================] - 7s 903us/step - loss: 0.6558 - acc: 0.7296
Epoch 7/80
8121/8121 [==============================] - 7s 894us/step - loss: 0.6139 - acc: 0.7468
Epoch 8/80
8121/8121 [==============================] - 7s 881us/step - loss: 0.6267 - acc: 0.7492
Epoch 9/80
8121/8121 [==============================] - 7s 920us/step - loss: 0.6032 - acc: 0.7570
Epoch 10/80
8121/8121 [==============================] - 7s 906us/step - loss: 0.5735 - acc: 0.7765
Epoch 11/80
8121/8121 [==============================] - 7s 900us/step - loss: 0.5388 - acc: 0.7939
Epoch 12/80
8121/8121 [==============================] - 7s 894us/step - loss: 0.5083 - acc: 0.8057
Epoch 1

...

Epoch 21/80
8121/8121 [==============================] - 8s 928us/step - loss: 0.4175 - acc: 0.8490
Epoch 22/80
8121/8121 [==============================] - 7s 914us/step - loss: 0.4120 - acc: 0.8485
Epoch 23/80
8121/8121 [==============================] - 7s 923us/step - loss: 0.4066 - acc: 0.8525
Epoch 24/80
8121/8121 [==============================] - 7s 915us/step - loss: 0.4019 - acc: 0.8549
Epoch 25/80
8121/8121 [==============================] - 7s 921us/step - loss: 0.4090 - acc: 0.8490
Epoch 26/80
8121/8121 [==============================] - 7s 895us/step - loss: 0.3890 - acc: 0.8595
Epoch 27/80
8121/8121 [==============================] - 8s 928us/step - loss: 0.3859 - acc: 0.8573
Epoch 28/80
8121/8121 [==============================] - 7s 910us/step - loss: 0.3762 - acc: 0.8616
Epoch 29/80
8121/8121 [==============================] - 7s 911us/step - loss: 0.3649 - acc: 0.8665
Epoch 30/80
8121/8121 [==============================] - 7s 903us/step - loss: 0.3627 - acc: 0.8629


8121/8121 [==============================] - 8s 952us/step - loss: 0.4092 - acc: 0.8500
Epoch 22/80
8121/8121 [==============================] - 8s 942us/step - loss: 0.4135 - acc: 0.8503
Epoch 23/80
8121/8121 [==============================] - 8s 941us/step - loss: 0.3959 - acc: 0.8549
Epoch 24/80
8121/8121 [==============================] - 8s 940us/step - loss: 0.3945 - acc: 0.8572
Epoch 25/80
8121/8121 [==============================] - 8s 944us/step - loss: 0.3836 - acc: 0.8611
Epoch 26/80
8121/8121 [==============================] - 8s 944us/step - loss: 0.3833 - acc: 0.8620
Epoch 27/80
8121/8121 [==============================] - 8s 945us/step - loss: 0.3902 - acc: 0.8602
Epoch 28/80
8121/8121 [==============================] - 8s 933us/step - loss: 0.3798 - acc: 0.8649
Epoch 29/80
8121/8121 [==============================] - 8s 936us/step - loss: 0.3671 - acc: 0.8690
Epoch 30/80
8121/8121 [==============================] - 8s 931us/step - loss: 0.3707 - acc: 0.8687
Epoch 31/80


[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed: 550.8min finished


Epoch 1/80
10151/10151 [==============================] - 18s 2ms/step - loss: 0.9767 - acc: 0.5798
Epoch 2/80
10151/10151 [==============================] - 10s 945us/step - loss: 0.8375 - acc: 0.6309
Epoch 3/80
10151/10151 [==============================] - 10s 946us/step - loss: 0.7310 - acc: 0.6933
Epoch 4/80
10151/10151 [==============================] - 10s 954us/step - loss: 0.6441 - acc: 0.7385
Epoch 5/80
10151/10151 [==============================] - 10s 950us/step - loss: 0.5892 - acc: 0.7664
Epoch 6/80
10151/10151 [==============================] - 10s 944us/step - loss: 0.5464 - acc: 0.7926
Epoch 7/80
10151/10151 [==============================] - 10s 956us/step - loss: 0.5335 - acc: 0.7970
Epoch 8/80
10151/10151 [==============================] - 10s 954us/step - loss: 0.5161 - acc: 0.8069
Epoch 9/80
10151/10151 [==============================] - 10s 945us/step - loss: 0.4969 - acc: 0.8116
Epoch 10/80
10151/10151 [==============================] - 10s 955us/step - loss: 0.

10151/10151 [==============================] - 10s 948us/step - loss: 0.2501 - acc: 0.9107
Finish training!


In [4]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean:%f (STD:%f) with: %r" % (mean, stdev, param))

Mean:0.842676 (STD:0.011239) with: {'batch_size': 70}
Mean:0.843562 (STD:0.004698) with: {'batch_size': 71}
Mean:0.842873 (STD:0.008904) with: {'batch_size': 72}
Mean:0.844843 (STD:0.010671) with: {'batch_size': 73}
Mean:0.843759 (STD:0.007495) with: {'batch_size': 74}
Mean:0.845729 (STD:0.007250) with: {'batch_size': 75}
Mean:0.839819 (STD:0.009618) with: {'batch_size': 76}
Mean:0.842774 (STD:0.008314) with: {'batch_size': 77}
Mean:0.844646 (STD:0.005512) with: {'batch_size': 78}
Mean:0.842971 (STD:0.008850) with: {'batch_size': 79}
Mean:0.844547 (STD:0.005648) with: {'batch_size': 80}


In [5]:
grid_result.best_score_

0.8457294902758298

In [6]:
grid_result.best_params_

{'batch_size': 75}